<a href="https://colab.research.google.com/github/BHatiru/NLP_Project/blob/main/LogReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
import glob
import torch
import pandas as pd

In [ ]:
train_paths = ['train/california_wildfires_2018_train.tsv',
               'train/canada_wildfires_2016_train.tsv',
               'train/cyclone_idai_2019_train.tsv',
               'train/ecuador_earthquake_2016_train.tsv',
               'train/greece_wildfires_2018_train.tsv',
               'train/hurricane_dorian_2019_train.tsv',
               'train/hurricane_florence_2018_train.tsv',
               'train/hurricane_harvey_2017_train.tsv',
               'train/hurricane_irma_2017_train.tsv',
               'train/hurricane_maria_2017_train.tsv',
               'train/hurricane_matthew_2016_train.tsv',
               'train/italy_earthquake_aug_2016_train.tsv',
               'train/kaikoura_earthquake_2016_train.tsv',
               'train/kerala_floods_2018_train.tsv',
               'train/maryland_floods_2018_train.tsv',
               'train/midwestern_us_floods_2019_train.tsv',
               'train/pakistan_earthquake_2019_train.tsv',
               'train/puebla_mexico_earthquake_2017_train.tsv',
               'train/srilanka_floods_2017_train.tsv',
               'dev/california_wildfires_2018_dev.tsv',
               'dev/canada_wildfires_2016_dev.tsv',
               'dev/cyclone_idai_2019_dev.tsv',
               'dev/ecuador_earthquake_2016_dev.tsv',
               'dev/greece_wildfires_2018_dev.tsv',
               'dev/hurricane_dorian_2019_dev.tsv',
               'dev/hurricane_florence_2018_dev.tsv',
               'dev/hurricane_harvey_2017_dev.tsv',
               'dev/hurricane_irma_2017_dev.tsv',
               'dev/hurricane_maria_2017_dev.tsv',
               'dev/hurricane_matthew_2016_dev.tsv',
               'dev/italy_earthquake_aug_2016_dev.tsv',
               'dev/kaikoura_earthquake_2016_dev.tsv',
               'dev/kerala_floods_2018_dev.tsv',
               'dev/maryland_floods_2018_dev.tsv',
               'dev/midwestern_us_floods_2019_dev.tsv',
               'dev/pakistan_earthquake_2019_dev.tsv',
               'dev/puebla_mexico_earthquake_2017_dev.tsv',
               'dev/srilanka_floods_2017_dev.tsv']
test_paths = ['test/california_wildfires_2018_test.tsv',
              'test/canada_wildfires_2016_test.tsv',
              'test/cyclone_idai_2019_test.tsv',
              'test/ecuador_earthquake_2016_test.tsv',
              'test/greece_wildfires_2018_test.tsv',
              'test/hurricane_dorian_2019_test.tsv',
              'test/hurricane_florence_2018_test.tsv',
              'test/hurricane_harvey_2017_test.tsv',
              'test/hurricane_irma_2017_test.tsv',
              'test/hurricane_maria_2017_test.tsv',
              'test/hurricane_matthew_2016_test.tsv',
              'test/italy_earthquake_aug_2016_test.tsv',
              'test/kaikoura_earthquake_2016_test.tsv',
              'test/kerala_floods_2018_test.tsv',
              'test/maryland_floods_2018_test.tsv',
              'test/midwestern_us_floods_2019_test.tsv',
              'test/pakistan_earthquake_2019_test.tsv',
              'test/puebla_mexico_earthquake_2017_test.tsv',
              'test/srilanka_floods_2017_test.tsv']
train_file_paths = []

for path in train_paths:
    files = glob.glob(path)
    train_file_paths.extend(files)

test_file_paths = []

for path in test_paths:
    files = glob.glob(path)
    test_file_paths.extend(files)

In [ ]:
train_dfs = []

for file in train_file_paths:
    df = pd.read_csv(file, sep='\t')
    df = df.iloc[:, 1:] # removing tweet ids
    df = df[df['class_label'] != 'missing_or_found_people']
    train_dfs.append(df)

train_df = pd.concat(train_dfs, ignore_index=True)

test_dfs = []

for file in test_file_paths:
    df = pd.read_csv(file, sep='\t')
    df = df.iloc[:, 1:] # removing tweet ids
    df = df[df['class_label'] != 'missing_or_found_people']
    test_dfs.append(df)

test_df = pd.concat(test_dfs, ignore_index=True)

In [ ]:
def reduce_class_size(df, max_samples_per_class=3000):
    return df.groupby('class_label').apply(lambda x: x.sample(min(len(x), max_samples_per_class))).reset_index(drop=True)

train_df = reduce_class_size(train_df)

In [ ]:
all_labels = pd.concat([train_df['class_label'], test_df['class_label']]).unique()

label_to_num = {label: num for num, label in enumerate(all_labels)}

train_df['class_label_num'] = train_df['class_label'].map(label_to_num)
test_df['class_label_num'] = test_df['class_label'].map(label_to_num)

In [ ]:
train_tweets = list(train_df.tweet_text)
train_labels = list(train_df.class_label_num)

test_tweets = list(test_df.tweet_text)
test_labels = list(test_df.class_label_num)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1, 1), use_idf=True)),
    ('clf', LogisticRegression(max_iter=1000, C=1, class_weight=None, penalty='l2', ))
])

In [ ]:
param_grid = {
    # 'tfidf__max_features': [500, 1000, 5000],
    # 'tfidf__ngram_range': [(1, 1), (1, 2)],
    # 'tfidf__use_idf': (True, False),
    # 'clf__C': [0.1, 1, 10, 100],
    # 'clf__penalty': ['l1', 'l2'],
    # 'clf__class_weight': [None, 'balanced']
}

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, n_jobs=-1)

In [ ]:
grid_search.fit(train_tweets, train_labels)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_features=5000,
                                                        stop_words='english')),
                                       ('clf',
                                        LogisticRegression(C=1,
                                                           max_iter=1000))]),
             n_jobs=-1, param_grid={}, verbose=2)

In [ ]:
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {}
Best Score: 0.7244129209585665


In [ ]:
test_preds = grid_search.predict(test_tweets)
print(accuracy_score(test_labels, test_preds))

0.711558854718982


In [ ]:
trains_preds = grid_search.predict(train_tweets)
print(accuracy_score(train_labels, trains_preds))

0.8211288653868261


In [ ]:
print(classification_report(test_labels, test_preds))

              precision    recall  f1-score   support

           0       0.58      0.72      0.64      1070
           1       0.82      0.90      0.86       790
           2       0.77      0.82      0.79      1617
           3       0.89      0.91      0.90      1447
           4       0.48      0.63      0.55      1245
           5       0.53      0.44      0.48      2407
           6       0.41      0.67      0.51       521
           7       0.87      0.73      0.80      4219
           8       0.80      0.75      0.77      1772

    accuracy                           0.71     15088
   macro avg       0.68      0.73      0.70     15088
weighted avg       0.73      0.71      0.71     15088

